### Chain和LCEL
一开始我们就说过，langchain不是市面上的唯一开发框架，虽然它现在非常火热算得上是不二之选，但是LLM应用开发这一年一直在不断发展，langchain也在不断发展，前面我们提到的各个组件，更多更像一个工具库：在其他框架也有这些工具，甚至你可以根据原理自己去写一个类似的出来。但是langchain独有的或者使得其能称为框架的，就是它的Chain，以及LCEL。

前面我们多次看到`|`这个符号，用它可以串联langchain的一些组件，组成一个叫Chain的东西。其实实际上，我们翻看上一集的ReActAgent的源码会发现，它其实也是一个chain
```python
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt
    | llm_with_stop
    | output_parser
)
```
有了之前的知识铺垫，我们来思考一个问题：我们知道这个市面上有各种模型，有些是通用模型，比如ChatGPT，有些是通过微调或者一开始训练的时候就带着目标的领域专用模型，比如金融的FinGPT以及代码编写目前火热的Codestral。如果我们希望用户的问题能被更合适的模型回答，而不是强行让一个模型回答所有问题，我们要如何设计？
很显然，首先我们要有个模型用于将问题分类，分类为比如金融，编码以及其他。然后我们又需要将金融的问题发送给FinGPT，将编码问题发送给Codestral。我们可以大概写出下面的伪代码：
```python
llm_default = ChatGPT()
llm_fin = FinGPT()
llm_code = CodeStral()

classify_prompt = '将用户的问题{question}分类为"Financial","Coding"和"Others"之一，只输出分类本身，不要输出其他文字'
classify_chain = classify_prompt | llm_default | EnumOutputParser
classify_result = classify_chain.invoke(question)
if classify_result == 'Financial':
  llm = llm_fin
elif classify_result == 'Coding':
  llm = llm_code
else:
  llm = llm_default
chain = prompt | llm | StrOutputParser
chain.invoke(question)
```

其实我们上面的代码已经使用了Chain了，我们多次使用了`|`去连接langchain的组件，如果不使用`|`，那么需要写成：
```python
LLMChain(llm=llm,prompt=prompt,memory=memory,parser=StrOutputParser)
```
其实从写的复杂程度来说两者都没太多差别，但是用`|`比较直观表达了数据从输入到流出的一个流向。当然这只是它的优点之一，关键的点在于所有可以用`\`链接的组件，都实现了一个统一的抽象接口：Runnable。那么反过来，任意你是自己实现的Runnable接口，也可以被接入到`|`中。只要是实现了Runnable接口，那么都会有一些方法：比如之前最常见到的invoke，还有stream，batch，异步等。同样的，因为Runnable是那么重要，langchain也提供了一些工具类用于辅助构建Runnable。

In [3]:
from libs.llm.qwen import qwen
from langchain_core.output_parsers import StrOutputParser

chain = qwen | StrOutputParser()
for chunk in chain.stream("介绍下你自己"):
    print(chunk, end='', flush=True)
chain.batch(['历史上今天发生了些啥大事', '介绍下你自己'])

['很抱歉，由于没有提供日期信息，我无法告诉您“历史上今天”发生了哪些大事。请提供具体的日期，我会很高兴为您提供相关信息。',
 '我是来自阿里云的大规模语言模型，我叫通义千问。我是阿里云自主研发的超大规模语言模型，也能够回答问题、创作文字，还能表达观点、撰写代码。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。']

#### RunnablePassthrough
RunnablePassthrough是一个简单的Runnable包装器，它主要目的是用于在链中传递数据，如果它出现在链的第一跳，它的值就是用户的输入，如果出现在中间，就代表上一跳的结果。可以回顾下RAG那个notebook中最后一段程序。

#### RunnableParallel
顾名思义，可以让多个链并发执行，尤其是当你分析你的链的某个环节不需要前后依赖可以并行的时候，就可以将相应部分用RunnableParallel包装其他，组成一个最终链

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

en_chain = ChatPromptTemplate.from_template("将我的问题{question}翻译为英语") | qwen | StrOutputParser()
jp_chain = (
        ChatPromptTemplate.from_template("将我的问题{question}翻译为日语") | qwen | StrOutputParser()
)

map_chain = RunnableParallel(en=en_chain, jp=jp_chain)

map_chain.invoke({"question": "生而为人我很抱歉"})

{'en': 'My question is, "As a human being, I am sorry."',
 'jp': '生まれたての問題として、人間であることを申し訳ありません。'}

##### RunnableBranch
从名字就可以知道，由于多个分支子链的情况，比如我们最开始假设的那种，需要根据用户问题选择更合适的模型去回答，我们那时候用if语句写了路由部分的代码，我们也可以将其替换成RunnableBranch
```python
ans_chain = RunnableBranch(
    (lambda x: x['classify'] == 'Financial', llm_fin),
    (lambda x: x['classify'] == 'Coding', llm_code),
    llm_default
)
final_chain = {"classify":classify_chain,"question":RunnablePassthrough()} | ans_chain
final_chain.invoke(question)
```

##### RunnableLambda
这个是一个很强大的包装器，可以将任何的普通的python函数或者可调用对象都转为Runnable，你就可以使用Runnable有的stream，batch，异步等功能

In [11]:
from langchain_core.runnables import RunnableLambda


def double(x):
    return x * 2


def add_one(x):
    return x + 1


runnable_double = RunnableLambda(double)
runnable_add_one = RunnableLambda(add_one)
final = runnable_double | runnable_add_one
final.batch(5)

[11, 11]

需要注意的是，RunnableLambda包装的函数都需要是单个参数的函数。有了这个Runnable，我们可以将各种各样的python函数集成到chain中，比如用RunnableLambda包装上报埋点，将链中间的值发送到采集系统。

##### Runnable简单原理

In [17]:
from abc import abstractmethod, ABC


class Runnable(ABC):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def __or__(self, other):
        def wrapper_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))

        return Runnable(wrapper_func)
    
    def invoke(self, *args, **kwargs):
        return self.func(*args, **kwargs)


def double(x):
    return x * 2


def add_one(x):
    return x + 1


chain = Runnable(add_one) | Runnable(double)
chain.invoke(10)

22